# Ground Truth Baseline - NYC Taxi Trip Analysis

This notebook computes comprehensive ground truth statistics from cleaned NYC taxi data.

**Computed Metrics:**
1. Temporal patterns (hourly/daily aggregates)
2. Fare distribution (percentiles, mean, std)
3. Spatial analytics (zone-based demand)
4. Trip duration statistics
5. Overall summary statistics

**Output:** JSON files for each metric category


## Background: Ground Truth Baseline Design

### Original proposal
- **Hadoop MapReduce (`ground_truth_mapper.py` / `ground_truth_reducer.py`)**:
  - Mapper read trip records line by line, parsed timestamps, fare, distance, and locations.
  - Emitted keys like `hour:HH`, `fare_bucket:X`, `location:ID`, `duration_stats` with values.
  - Reducer grouped by key and computed stats (count, mean, median, stdev, percentiles) to build a **ground truth baseline** for:
    - Temporal patterns
    - Fare distribution
    - Location demand
    - Trip duration statistics.

- **"Better approach" Spark job (`ground_truth_baseline.py`)**:
  - Spark read parquet data from S3 (`s3://{bucket}-processed/clean-data/...`).
  - Added time features: `hour`, `day_of_week`, `trip_duration_min`.
  - Wrote multiple JSON outputs to S3:
    - `hourly_stats/`
    - `fare_distribution/`
    - `zone_demand/`
    - `duration_stats/`
    - `summary/` (overall summary per year), intended to run on EMR via `spark-submit`.

### What this notebook implements (pandas version of Spark job)
- **Input**: cleaned parquet files from `/Users/gouravdhama/Documents/bubu/cleaned_data`, read via `safe_read_parquet` (handles timestamp overflow).
- **Automatic schema detection**:
  - Pickup/dropoff datetime
  - Fare amount
  - Trip distance
  - `PULocationID` / `DOLocationID`
  - `passenger_count` (when available).
- **Per-file metrics computed**:
  - Temporal patterns (hourly + daily) similar to the Spark design.
  - Fare distribution (mean/median/std + percentiles + fare buckets) when a fare column exists.
  - Spatial analytics (top pickup zones, dropoff zones, and OD pairs) when location IDs are present.
  - Trip duration stats when both pickup and dropoff datetimes are present.
  - Overall summary (total trips, total revenue, total distance, avg fare, avg distance, unique zones).
- **Per-file output**: each file saves a JSON under `ground_truth_results/summary/*_ground_truth.json`.
- **Aggregations by file type** (`fhv`, `fhvhv`, `green`, `yellow`):
  - `ground_truth_results/hourly_stats/{type}_aggregated.json`
  - `ground_truth_results/summary/{type}_overall_summary.json`.
- **Master catalog**: `ground_truth_results/MASTER_INDEX.json` describes all processed files, available metrics, and directory structure.

### Example aggregate results from the last run
- **FHV (24 files)**:
  - Total trips: **49,570,306** (no revenue/avg fare because FHV schema has no fare/distance columns).
- **FHVHV (22 files)**:
  - Total trips: **441,793,115**.
- **GREEN (26 files)**:
  - Total trips: **7,929,571**
  - Total revenue: **$107,013,181.87**
  - Average fare: **$17.19**.
- **YELLOW (28 files)**:
  - Total trips: **161,260,245**
  - Total revenue: **$2,190,767,446.07**
  - Average fare: **$18.19**.

These numbers form your **ground truth baseline** that future streaming/online algorithms (e.g., MSK-based pipelines) can be compared against.


## 1. Setup and Configuration


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Directories
CLEANED_DATA_DIR = "/Users/gouravdhama/Documents/bubu/cleaned_data"
RESULTS_DIR = "/Users/gouravdhama/Documents/bubu/ground_truth_results"

# Create results directory structure
Path(RESULTS_DIR).mkdir(exist_ok=True)
for subdir in ['hourly_stats', 'daily_stats', 'fare_distribution', 'zone_demand', 'duration_stats', 'summary']:
    Path(f"{RESULTS_DIR}/{subdir}").mkdir(exist_ok=True)

print("✓ Directories setup complete")
print(f"  Cleaned data: {CLEANED_DATA_DIR}")
print(f"  Results: {RESULTS_DIR}")


✓ Directories setup complete
  Cleaned data: /Users/gouravdhama/Documents/bubu/cleaned_data
  Results: /Users/gouravdhama/Documents/bubu/ground_truth_results


## 2. Helper Functions


In [2]:
import pyarrow.parquet as pq

def safe_read_parquet(path: Path) -> pd.DataFrame:
    """Read parquet file with timestamp overflow handling."""
    try:
        return pd.read_parquet(path)
    except Exception as e:
        error_type = type(e).__name__
        try:
            msg = str(e)
        except Exception:
            msg = ""
        
        is_ts_error = (
            'ArrowInvalid' in error_type or
            'out of bounds' in msg.lower() or
            'timestamp' in msg.lower() or
            'casting' in msg.lower()
        )
        
        if not is_ts_error:
            raise
        
        print(f"    ⚠️  Timestamp issue for {path.name}, using safe mode...")
        table = pq.read_table(path)
        df = table.to_pandas(timestamp_as_object=True)
        
        # Convert datetime-like columns
        for col in df.columns:
            if df[col].dtype == 'object' and any(x in col.lower() for x in ['time', 'datetime', 'date']):
                try:
                    df[col] = pd.to_datetime(df[col], errors='coerce')
                except Exception:
                    pass
        return df

def compute_percentiles(series, percentiles=[0.25, 0.5, 0.75, 0.90, 0.95, 0.99]):
    """Compute percentiles for a series."""
    if len(series) == 0:
        return {f"p{int(p*100)}": None for p in percentiles}
    return {f"p{int(p*100)}": float(series.quantile(p)) for p in percentiles}

def save_json(data, filepath):
    """Save data to JSON file."""
    with open(filepath, 'w') as f:
        json.dump(data, f, indent=2, default=str)
    print(f"  ✓ Saved: {filepath}")

print("✓ Helper functions loaded")


✓ Helper functions loaded


## 3. Column Identification Functions


In [3]:
def identify_datetime_columns(df):
    """Identify datetime columns (pickup and dropoff)."""
    datetime_cols = {}
    
    # Common patterns for pickup datetime
    pickup_patterns = ['pickup_datetime', 'tpep_pickup_datetime', 'lpep_pickup_datetime', 'pickup_dt']
    for col in df.columns:
        if any(pattern in col.lower() for pattern in pickup_patterns):
            if pd.api.types.is_datetime64_any_dtype(df[col]) or df[col].dtype == 'object':
                datetime_cols['pickup'] = col
                break
    
    # Common patterns for dropoff datetime
    dropoff_patterns = ['dropoff_datetime', 'tpep_dropoff_datetime', 'lpep_dropoff_datetime', 'dropoff_dt']
    for col in df.columns:
        if any(pattern in col.lower() for pattern in dropoff_patterns):
            if pd.api.types.is_datetime64_any_dtype(df[col]) or df[col].dtype == 'object':
                datetime_cols['dropoff'] = col
                break
    
    return datetime_cols

def identify_columns(df):
    """Identify relevant columns for analysis."""
    cols = {
        'datetime': identify_datetime_columns(df),
        'fare': None,
        'distance': None,
        'passenger_count': None,
        'pu_location': None,
        'do_location': None
    }
    
    # Find fare column
    for col in df.columns:
        if 'fare' in col.lower() and 'amount' in col.lower():
            cols['fare'] = col
            break
    
    # Find distance column
    for col in df.columns:
        if 'distance' in col.lower() and 'trip' in col.lower():
            cols['distance'] = col
            break
    
    # Find passenger count
    for col in df.columns:
        if 'passenger' in col.lower():
            cols['passenger_count'] = col
            break
    
    # Find pickup location
    for col in df.columns:
        if 'pulocation' in col.lower() or ('pickup' in col.lower() and 'location' in col.lower()):
            cols['pu_location'] = col
            break
    
    # Find dropoff location
    for col in df.columns:
        if 'dolocation' in col.lower() or ('dropoff' in col.lower() and 'location' in col.lower()):
            cols['do_location'] = col
            break
    
    return cols

print("✓ Column identification functions loaded")


✓ Column identification functions loaded


## 4. Ground Truth Computation Functions


In [4]:
def compute_temporal_patterns(df, cols, file_type):
    """Compute hourly and daily temporal patterns."""
    results = {}
    
    pickup_col = cols['datetime'].get('pickup')
    fare_col = cols['fare']
    distance_col = cols['distance']
    
    if not pickup_col:
        return None
    
    # Ensure datetime type
    df[pickup_col] = pd.to_datetime(df[pickup_col], errors='coerce')
    df = df.dropna(subset=[pickup_col])
    
    if len(df) == 0:
        return None
    
    # Add time features
    df['hour'] = df[pickup_col].dt.hour
    df['day_of_week'] = df[pickup_col].dt.dayofweek
    df['day_name'] = df[pickup_col].dt.day_name()
    
    # Compute trip duration if both pickup and dropoff exist
    dropoff_col = cols['datetime'].get('dropoff')
    if dropoff_col:
        df[dropoff_col] = pd.to_datetime(df[dropoff_col], errors='coerce')
        df['trip_duration_min'] = (df[dropoff_col] - df[pickup_col]).dt.total_seconds() / 60.0
        df['trip_duration_min'] = df['trip_duration_min'].clip(lower=0, upper=300)  # Cap at 5 hours
    
    # HOURLY STATS
    hourly_agg = {'trip_count': ('hour', 'size')}
    if fare_col:
        hourly_agg.update({
            'avg_fare': (fare_col, 'mean'),
            'median_fare': (fare_col, 'median'),
            'std_fare': (fare_col, 'std')
        })
    if distance_col:
        hourly_agg['avg_distance'] = (distance_col, 'mean')
    if 'trip_duration_min' in df.columns:
        hourly_agg['avg_duration_min'] = ('trip_duration_min', 'mean')
    
    hourly_stats = df.groupby('hour').agg(**hourly_agg).reset_index()
    results['hourly'] = hourly_stats.to_dict(orient='records')
    
    # DAILY STATS
    daily_agg = {'trip_count': ('day_of_week', 'size')}
    if fare_col:
        daily_agg.update({
            'avg_fare': (fare_col, 'mean'),
            'total_revenue': (fare_col, 'sum')
        })
    
    daily_stats = df.groupby(['day_of_week', 'day_name']).agg(**daily_agg).reset_index()
    results['daily'] = daily_stats.to_dict(orient='records')
    
    return results

print("✓ Temporal pattern computation loaded")


✓ Temporal pattern computation loaded


In [5]:
def compute_fare_distribution(df, cols):
    """Compute fare distribution statistics."""
    fare_col = cols['fare']
    
    if not fare_col or fare_col not in df.columns:
        return None
    
    fares = df[fare_col].dropna()
    
    if len(fares) == 0:
        return None
    
    stats = {
        'count': int(len(fares)),
        'mean': float(fares.mean()),
        'median': float(fares.median()),
        'std': float(fares.std()),
        'min': float(fares.min()),
        'max': float(fares.max()),
        **compute_percentiles(fares)
    }
    
    # Fare buckets (histogram)
    fare_buckets = pd.cut(fares, bins=[0, 10, 20, 30, 40, 50, 100, 500], 
                          labels=['0-10', '10-20', '20-30', '30-40', '40-50', '50-100', '100+'])
    stats['distribution'] = fare_buckets.value_counts().sort_index().to_dict()
    
    return stats

print("✓ Fare distribution computation loaded")


✓ Fare distribution computation loaded


In [6]:
def compute_spatial_analytics(df, cols):
    """Compute zone-based demand and spatial patterns."""
    pu_col = cols['pu_location']
    do_col = cols['do_location']
    fare_col = cols['fare']
    
    results = {}
    
    # Pickup zone demand
    if pu_col and pu_col in df.columns:
        pu_agg = {'pickup_count': (pu_col, 'size')}
        if fare_col:
            pu_agg['avg_fare_from_zone'] = (fare_col, 'mean')
        
        zone_demand = df.groupby(pu_col).agg(**pu_agg).reset_index()
        zone_demand = zone_demand.sort_values('pickup_count', ascending=False).head(50)
        results['top_pickup_zones'] = zone_demand.to_dict(orient='records')
    
    # Dropoff zone demand
    if do_col and do_col in df.columns:
        do_demand = df.groupby(do_col).size().reset_index(name='dropoff_count')
        do_demand = do_demand.sort_values('dropoff_count', ascending=False).head(50)
        results['top_dropoff_zones'] = do_demand.to_dict(orient='records')
    
    # OD pairs (Origin-Destination)
    if pu_col and do_col and pu_col in df.columns and do_col in df.columns:
        od_pairs = df.groupby([pu_col, do_col]).size().reset_index(name='trip_count')
        od_pairs = od_pairs.sort_values('trip_count', ascending=False).head(100)
        results['top_od_pairs'] = od_pairs.to_dict(orient='records')
    
    return results if results else None

print("✓ Spatial analytics computation loaded")


✓ Spatial analytics computation loaded


In [7]:
def compute_duration_stats(df, cols):
    """Compute trip duration statistics."""
    pickup_col = cols['datetime'].get('pickup')
    dropoff_col = cols['datetime'].get('dropoff')
    
    if not pickup_col or not dropoff_col:
        return None
    
    df[pickup_col] = pd.to_datetime(df[pickup_col], errors='coerce')
    df[dropoff_col] = pd.to_datetime(df[dropoff_col], errors='coerce')
    
    df['trip_duration_min'] = (df[dropoff_col] - df[pickup_col]).dt.total_seconds() / 60.0
    
    # Filter valid durations (0 to 300 minutes)
    durations = df['trip_duration_min'].dropna()
    durations = durations[(durations > 0) & (durations <= 300)]
    
    if len(durations) == 0:
        return None
    
    stats = {
        'count': int(len(durations)),
        'mean_duration_min': float(durations.mean()),
        'median_duration_min': float(durations.median()),
        'std_duration_min': float(durations.std()),
        'min_duration_min': float(durations.min()),
        'max_duration_min': float(durations.max()),
        **compute_percentiles(durations)
    }
    
    return stats

def compute_summary_stats(df, cols, file_name):
    """Compute overall summary statistics."""
    stats = {
        'file_name': file_name,
        'total_trips': len(df)
    }
    
    # Unique zones
    if cols['pu_location'] and cols['pu_location'] in df.columns:
        stats['unique_pickup_zones'] = int(df[cols['pu_location']].nunique())
    if cols['do_location'] and cols['do_location'] in df.columns:
        stats['unique_dropoff_zones'] = int(df[cols['do_location']].nunique())
    
    # Revenue and distance
    if cols['fare'] and cols['fare'] in df.columns:
        stats['total_revenue'] = float(df[cols['fare']].sum())
        stats['avg_fare'] = float(df[cols['fare']].mean())
    
    if cols['distance'] and cols['distance'] in df.columns:
        stats['total_distance'] = float(df[cols['distance']].sum())
        stats['avg_distance'] = float(df[cols['distance']].mean())
    
    # Passenger count
    if cols['passenger_count'] and cols['passenger_count'] in df.columns:
        stats['avg_passenger_count'] = float(df[cols['passenger_count']].mean())
    
    return stats

print("✓ Duration & summary stats computation loaded")


✓ Duration & summary stats computation loaded


## 5. Main Processing Function


In [8]:
def process_file(file_path):
    """Process a single parquet file and compute all ground truth metrics."""
    file_name = file_path.name
    
    # Determine file type
    if 'yellow' in file_name:
        file_type = 'yellow'
    elif 'green' in file_name:
        file_type = 'green'
    elif 'fhvhv' in file_name:
        file_type = 'fhvhv'
    elif 'fhv' in file_name:
        file_type = 'fhv'
    else:
        file_type = 'unknown'
    
    print(f"\n📊 Processing: {file_name} (type: {file_type})")
    
    # Read file
    try:
        df = safe_read_parquet(file_path)
        print(f"  Rows: {len(df):,}, Cols: {len(df.columns)}")
    except Exception as e:
        print(f"  ❌ Error reading file: {e}")
        return None
    
    if len(df) == 0:
        print(f"  ⚠️  Empty dataframe, skipping...")
        return None
    
    # Identify columns
    cols = identify_columns(df)
    print(f"  Identified columns:")
    print(f"    Pickup: {cols['datetime'].get('pickup')}")
    print(f"    Fare: {cols['fare']}")
    print(f"    Distance: {cols['distance']}")
    print(f"    PU Location: {cols['pu_location']}")
    
    # Compute metrics
    results = {
        'file_name': file_name,
        'file_type': file_type,
        'processed_at': datetime.now().isoformat()
    }
    
    # 1. Temporal patterns
    temporal = compute_temporal_patterns(df.copy(), cols, file_type)
    if temporal:
        results['temporal_patterns'] = temporal
        print(f"  ✓ Temporal patterns computed")
    
    # 2. Fare distribution
    fare_dist = compute_fare_distribution(df.copy(), cols)
    if fare_dist:
        results['fare_distribution'] = fare_dist
        print(f"  ✓ Fare distribution computed")
    
    # 3. Spatial analytics
    spatial = compute_spatial_analytics(df.copy(), cols)
    if spatial:
        results['spatial_analytics'] = spatial
        print(f"  ✓ Spatial analytics computed")
    
    # 4. Duration stats
    duration = compute_duration_stats(df.copy(), cols)
    if duration:
        results['duration_stats'] = duration
        print(f"  ✓ Duration stats computed")
    
    # 5. Summary
    summary = compute_summary_stats(df.copy(), cols, file_name)
    results['summary'] = summary
    print(f"  ✓ Summary stats computed")
    
    return results

print("✓ Main processing function loaded")


✓ Main processing function loaded


## 6. Execute Ground Truth Computation

Run this cell to process all cleaned parquet files and compute ground truth metrics.


In [9]:
# Get all cleaned parquet files
cleaned_files = sorted(Path(CLEANED_DATA_DIR).glob("*.parquet"))

print(f"\n{'='*80}")
print(f"GROUND TRUTH BASELINE COMPUTATION")
print(f"{'='*80}")
print(f"Total files: {len(cleaned_files)}")
print(f"Output directory: {RESULTS_DIR}")
print(f"{'='*80}")

# Process all files
all_results = []
file_type_groups = defaultdict(list)

for idx, file_path in enumerate(cleaned_files, 1):
    print(f"\n[{idx}/{len(cleaned_files)}] {file_path.name}")
    
    result = process_file(file_path)
    
    if result:
        all_results.append(result)
        file_type = result['file_type']
        file_type_groups[file_type].append(result)
        
        # Save individual file result
        output_file = f"{RESULTS_DIR}/summary/{file_path.stem}_ground_truth.json"
        save_json(result, output_file)

print(f"\n{'='*80}")
print(f"✅ PROCESSING COMPLETE")
print(f"{'='*80}")
print(f"Successfully processed: {len(all_results)}/{len(cleaned_files)} files")
print(f"\nFiles by type:")
for file_type, results in file_type_groups.items():
    print(f"  {file_type}: {len(results)} files")



GROUND TRUTH BASELINE COMPUTATION
Total files: 100
Output directory: /Users/gouravdhama/Documents/bubu/ground_truth_results

[1/100] fhv_tripdata_2015-02.parquet

📊 Processing: fhv_tripdata_2015-02.parquet (type: fhv)
  Rows: 3,053,183, Cols: 6
  Identified columns:
    Pickup: pickup_datetime
    Fare: None
    Distance: None
    PU Location: PUlocationID
  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhv_tripdata_2015-02_ground_truth.json

[2/100] fhv_tripdata_2015-12.parquet

📊 Processing: fhv_tripdata_2015-12.parquet (type: fhv)
  Rows: 8,888,809, Cols: 6
  Identified columns:
    Pickup: pickup_datetime
    Fare: None
    Distance: None
    PU Location: PUlocationID
  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhv_tripdata

  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhv_tripdata_2025-02_ground_truth.json

[19/100] fhv_tripdata_2025-03.parquet

📊 Processing: fhv_tripdata_2025-03.parquet (type: fhv)
  Rows: 2,182,992, Cols: 6
  Identified columns:
    Pickup: pickup_datetime
    Fare: None
    Distance: None
    PU Location: PUlocationID
  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhv_tripdata_2025-03_ground_truth.json

[20/100] fhv_tripdata_2025-04.parquet

📊 Processing: fhv_tripdata_2025-04.parquet (type: fhv)
  Rows: 1,699,478, Cols: 6
  Identified columns:
    Pickup: pickup_datetime
    Fare: None
    Distance: None
    PU Location: PUlocationID
  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Du

  Rows: 20,028,282, Cols: 24
  Identified columns:
    Pickup: pickup_datetime
    Fare: None
    Distance: None
    PU Location: PULocationID
  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhvhv_tripdata_2024-10_ground_truth.json

[37/100] fhvhv_tripdata_2024-11.parquet

📊 Processing: fhvhv_tripdata_2024-11.parquet (type: fhvhv)
  Rows: 19,987,533, Cols: 24
  Identified columns:
    Pickup: pickup_datetime
    Fare: None
    Distance: None
    PU Location: PULocationID
  ✓ Temporal patterns computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhvhv_tripdata_2024-11_ground_truth.json

[38/100] fhvhv_tripdata_2024-12.parquet

📊 Processing: fhvhv_tripdata_2024-12.parquet (type: fhvhv)
  Rows: 21,068,851, Cols: 24
  Identified column

  ✓ Fare distribution computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/green_tripdata_2024-01_ground_truth.json

[54/100] green_tripdata_2024-02.parquet

📊 Processing: green_tripdata_2024-02.parquet (type: green)
  Rows: 52,835, Cols: 19
  Identified columns:
    Pickup: lpep_pickup_datetime
    Fare: fare_amount
    Distance: trip_distance
    PU Location: PULocationID
  ✓ Temporal patterns computed
  ✓ Fare distribution computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/green_tripdata_2024-02_ground_truth.json

[55/100] green_tripdata_2024-03.parquet

📊 Processing: green_tripdata_2024-03.parquet (type: green)
  Rows: 56,681, Cols: 19
  Identified columns:
    Pickup: lpep_pickup_datetime
    Fare: fare_amount
    Distance: trip_distance
    PU Locati

  Rows: 45,475, Cols: 20
  Identified columns:
    Pickup: lpep_pickup_datetime
    Fare: fare_amount
    Distance: trip_distance
    PU Location: PULocationID
  ✓ Temporal patterns computed
  ✓ Fare distribution computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/green_tripdata_2025-08_ground_truth.json

[73/100] yellow_tripdata_2009-02.parquet

📊 Processing: yellow_tripdata_2009-02.parquet (type: yellow)
  Rows: 13,380,122, Cols: 16
  Identified columns:
    Pickup: Trip_Pickup_DateTime
    Fare: None
    Distance: Trip_Distance
    PU Location: None
  ✓ Temporal patterns computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/yellow_tripdata_2009-02_ground_truth.json

[74/100] yellow_tripdata_2009-12.parquet

📊 Processing: yellow_tripdata_2009-12.parquet (type: yellow)
  Rows: 14,583,404

  ✓ Temporal patterns computed
  ✓ Fare distribution computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/yellow_tripdata_2024-08_ground_truth.json

[89/100] yellow_tripdata_2024-09.parquet

📊 Processing: yellow_tripdata_2024-09.parquet (type: yellow)
  Rows: 3,529,402, Cols: 19
  Identified columns:
    Pickup: tpep_pickup_datetime
    Fare: fare_amount
    Distance: trip_distance
    PU Location: PULocationID
  ✓ Temporal patterns computed
  ✓ Fare distribution computed
  ✓ Spatial analytics computed
  ✓ Duration stats computed
  ✓ Summary stats computed
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/yellow_tripdata_2024-09_ground_truth.json

[90/100] yellow_tripdata_2024-10.parquet

📊 Processing: yellow_tripdata_2024-10.parquet (type: yellow)
  Rows: 3,722,475, Cols: 19
  Identified columns:
    Pickup: tpep_pickup_datetime
    Fare: fare_amo

## 7. Aggregate Results by File Type

This cell aggregates results across all files of the same type (yellow, green, fhv, fhvhv).


In [11]:
print(f"\n{'='*80}")
print("AGGREGATING RESULTS BY FILE TYPE")
print(f"{'='*80}")

for file_type, results in file_type_groups.items():
    print(f"\n📈 Aggregating {file_type.upper()} files ({len(results)} files)...")
    
    # Aggregate hourly stats
    all_hourly = []
    for r in results:
        if 'temporal_patterns' in r and 'hourly' in r['temporal_patterns']:
            all_hourly.extend(r['temporal_patterns']['hourly'])
    
    if all_hourly:
        hourly_df = pd.DataFrame(all_hourly)
        
        # Build aggregation dict based on available columns
        agg_dict = {'trip_count': 'sum'}
        if 'avg_fare' in hourly_df.columns:
            agg_dict['avg_fare'] = 'mean'
        if 'avg_distance' in hourly_df.columns:
            agg_dict['avg_distance'] = 'mean'
        if 'avg_duration_min' in hourly_df.columns:
            agg_dict['avg_duration_min'] = 'mean'
        
        agg_hourly = hourly_df.groupby('hour').agg(agg_dict).reset_index()
        
        output_file = f"{RESULTS_DIR}/hourly_stats/{file_type}_aggregated.json"
        save_json(agg_hourly.to_dict(orient='records'), output_file)
    
    # Aggregate summary stats
    summaries = [r['summary'] for r in results if 'summary' in r]
    if summaries:
        total_summary = {
            'file_type': file_type,
            'total_files': len(summaries),
            'total_trips': sum(s.get('total_trips', 0) for s in summaries)
        }
        
        # Add optional fields only if they exist
        revenue_values = [s.get('total_revenue', 0) for s in summaries if 'total_revenue' in s]
        if revenue_values:
            total_summary['total_revenue'] = sum(revenue_values)
        
        distance_values = [s.get('total_distance', 0) for s in summaries if 'total_distance' in s]
        if distance_values:
            total_summary['total_distance'] = sum(distance_values)
        
        avg_fare_values = [s.get('avg_fare') for s in summaries if 'avg_fare' in s]
        if avg_fare_values:
            total_summary['avg_fare'] = np.mean(avg_fare_values)
        
        avg_distance_values = [s.get('avg_distance') for s in summaries if 'avg_distance' in s]
        if avg_distance_values:
            total_summary['avg_distance'] = np.mean(avg_distance_values)
        
        output_file = f"{RESULTS_DIR}/summary/{file_type}_overall_summary.json"
        save_json(total_summary, output_file)
        
        print(f"  Total trips: {total_summary['total_trips']:,}")
        if 'total_revenue' in total_summary:
            print(f"  Total revenue: ${total_summary['total_revenue']:,.2f}")
        if 'avg_fare' in total_summary:
            print(f"  Avg fare: ${total_summary['avg_fare']:.2f}")

print(f"\n✅ Aggregation complete!")



AGGREGATING RESULTS BY FILE TYPE

📈 Aggregating FHV files (24 files)...
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/hourly_stats/fhv_aggregated.json
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhv_overall_summary.json
  Total trips: 49,570,306

📈 Aggregating FHVHV files (22 files)...
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/hourly_stats/fhvhv_aggregated.json
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/fhvhv_overall_summary.json
  Total trips: 441,793,115

📈 Aggregating GREEN files (26 files)...
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/hourly_stats/green_aggregated.json
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/summary/green_overall_summary.json
  Total trips: 7,929,571
  Total revenue: $107,013,181.87
  Avg fare: $17.19

📈 Aggregating YELLOW files (28 files)...
  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/hourl

## 8. Save Master Index


In [12]:
# Create master index of all results
master_index = {
    'generated_at': datetime.now().isoformat(),
    'total_files_processed': len(all_results),
    'file_types': {ft: len(results) for ft, results in file_type_groups.items()},
    'output_structure': {
        'hourly_stats': 'Aggregated hourly statistics by file type',
        'daily_stats': 'Aggregated daily statistics by file type',
        'fare_distribution': 'Fare distribution metrics',
        'zone_demand': 'Spatial analytics and zone-based demand',
        'duration_stats': 'Trip duration statistics',
        'summary': 'Individual file summaries and overall aggregates'
    },
    'files': [{
        'file_name': r['file_name'],
        'file_type': r['file_type'],
        'total_trips': r['summary']['total_trips'],
        'has_temporal': 'temporal_patterns' in r,
        'has_fare': 'fare_distribution' in r,
        'has_spatial': 'spatial_analytics' in r,
        'has_duration': 'duration_stats' in r
    } for r in all_results]
}

master_index_file = f"{RESULTS_DIR}/MASTER_INDEX.json"
save_json(master_index, master_index_file)

print(f"\n{'='*80}")
print("✅ GROUND TRUTH BASELINE COMPUTATION COMPLETE")
print(f"{'='*80}")
print(f"\nResults saved to: {RESULTS_DIR}")
print(f"\nTo view master index:")
print(f"  cat {master_index_file}")
print(f"\nDirectory structure:")
print(f"  {RESULTS_DIR}/")
print(f"    ├── hourly_stats/        # Hourly aggregated metrics")
print(f"    ├── daily_stats/         # Daily aggregated metrics")
print(f"    ├── fare_distribution/   # Fare percentiles and distributions")
print(f"    ├── zone_demand/         # Spatial analytics")
print(f"    ├── duration_stats/      # Trip duration statistics")
print(f"    ├── summary/             # Individual and overall summaries")
print(f"    └── MASTER_INDEX.json    # Master catalog of all results")


  ✓ Saved: /Users/gouravdhama/Documents/bubu/ground_truth_results/MASTER_INDEX.json

✅ GROUND TRUTH BASELINE COMPUTATION COMPLETE

Results saved to: /Users/gouravdhama/Documents/bubu/ground_truth_results

To view master index:
  cat /Users/gouravdhama/Documents/bubu/ground_truth_results/MASTER_INDEX.json

Directory structure:
  /Users/gouravdhama/Documents/bubu/ground_truth_results/
    ├── hourly_stats/        # Hourly aggregated metrics
    ├── daily_stats/         # Daily aggregated metrics
    ├── fare_distribution/   # Fare percentiles and distributions
    ├── zone_demand/         # Spatial analytics
    ├── duration_stats/      # Trip duration statistics
    ├── summary/             # Individual and overall summaries
    └── MASTER_INDEX.json    # Master catalog of all results


## 9. Display Sample Results

View a sample of the computed ground truth metrics.


In [13]:
# Display sample from one file
if all_results:
    sample = all_results[0]
    print(f"\n{'='*80}")
    print(f"SAMPLE RESULTS: {sample['file_name']}")
    print(f"{'='*80}")
    
    # Summary
    print(f"\n📊 SUMMARY:")
    for key, value in sample['summary'].items():
        if isinstance(value, (int, float)):
            if 'revenue' in key or 'fare' in key:
                print(f"  {key}: ${value:,.2f}")
            elif isinstance(value, int):
                print(f"  {key}: {value:,}")
            else:
                print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")
    
    # Hourly pattern (first 5 hours)
    if 'temporal_patterns' in sample and 'hourly' in sample['temporal_patterns']:
        print(f"\n⏰ HOURLY PATTERNS (sample):")
        hourly_df = pd.DataFrame(sample['temporal_patterns']['hourly'][:5])
        print(hourly_df.to_string(index=False))
    
    # Fare distribution
    if 'fare_distribution' in sample:
        print(f"\n💵 FARE DISTRIBUTION:")
        fare = sample['fare_distribution']
        print(f"  Mean: ${fare.get('mean', 0):.2f}")
        print(f"  Median: ${fare.get('median', 0):.2f}")
        print(f"  P95: ${fare.get('p95', 0):.2f}")
        print(f"  Min: ${fare.get('min', 0):.2f}")
        print(f"  Max: ${fare.get('max', 0):.2f}")



SAMPLE RESULTS: fhv_tripdata_2015-02.parquet

📊 SUMMARY:
  file_name: fhv_tripdata_2015-02.parquet
  total_trips: 3,053,183
  unique_pickup_zones: 263
  unique_dropoff_zones: 2

⏰ HOURLY PATTERNS (sample):
 hour  trip_count  avg_duration_min
    0      109701          0.002604
    1       71390          0.000727
    2       49220          0.001076
    3       37350          0.000779
    4       41050          0.000000
